## Import cac th vien

In [1]:
from datetime import datetime, timedelta
import pandas as pd
import glob
import os
from pathlib import Path
from xlwings.constants import AutoFillType
from main_function import convert_string

In [2]:
to_day = datetime.now()
w_num= str(datetime.strftime(datetime.now(),'%V'))
d_thang = datetime.strftime(datetime.now(),'%m')
d_nam = str(datetime.strftime(datetime.now(),'%Y'))
day_of_last_week = (to_day - timedelta(days=6)).strftime('%d')
month_of_last_week = (to_day - timedelta(days=6)).strftime('%m')
# print(f'tuần {w_num} thang {d_thang} năm {d_nam}'


# Cách 2 để lấy week num

to_day1 = datetime.today()
w_num2 = to_day1.isocalendar().week #cach nay cung duoc
w_num2 


In [3]:
ddd = input('nhap duong dan toi folder chua file bao cao che bien')
bao_cao = r'D:\1 Workbench\0 Code_bc tuan\BC_chebien.xlsx'

## Tao tu dien doi ten giong

In [4]:
ten = pd.read_excel(bao_cao,sheet_name='Sheet1', usecols='A:B',skiprows=1)
ten = ten.dropna()
ten.columns=['id','name']

In [5]:
ten = ten.astype({'id':'str','name':'str'})

In [6]:
id = ten['id'].to_list()
name = ten['name'].to_list()
name_dict = dict(zip(id, name))


## Doc du lieu vao data frame

In [7]:
duong_dan = Path(ddd)

# ex_file = glob.glob(os.path.join(duong_dan, "*.xlsx"))
ex_file = glob.glob(os.path.join(duong_dan,'*.xls*'))

In [8]:
df_file = []


In [9]:
# lap qua tat ca cac file excel
for f in ex_file:
    if f.split("\\")[-1] != "data_out_put.xlsx":
        df = pd.read_excel(f, sheet_name='data',converters={'ngay_che_bien':str,'ngay_nhap_kho ':str})
        df_file.append(df)
        # see pd.concat documentation for more info

    # write DataFrame to an excel sheet
    # appended_data.to_excel('appended.xls)
        # print('Location:', f)
        # print('File name:', f.split("\\")[-1])
        # print('Content:')
        # display(df)
    data = pd.concat(df_file,axis=0, ignore_index=True)
    

In [10]:
data =data.dropna(subset =['Ten_giong'])

In [11]:
data['ID'] = [s.strip()[-10:][3:5] for s in data['MSL']]

In [12]:
data['MSL'] = [s.replace('Đ', 'D') for s in data['MSL']]

In [13]:
data['Ten_giong'] = data['Ten_giong'].apply(convert_string)

# nhom

In [14]:
gk = data.groupby(['NM','Ten_giong'], group_keys=False).agg({'Ten_giong':'count','kl_truoc_che_bien':'sum','kl_thanh_pham':'sum','klpp1':'sum','kl_pp2':'sum','kl_pp3':'sum','kl_hao_hut':'sum'})


In [15]:
gk['TL_thanh_pham'] = gk['kl_thanh_pham']/gk['kl_truoc_che_bien']*100


In [16]:
gk['TL_hao_hut'] = gk['kl_hao_hut']/gk['kl_truoc_che_bien']*100

In [17]:
gk = gk.reindex(columns=['Ten_giong','kl_truoc_che_bien', 'kl_thanh_pham', 'TL_thanh_pham',
       'klpp1', 'kl_pp2', 'kl_pp3', 'kl_hao_hut', 'TL_hao_hut'])

In [18]:
gk.rename(columns={'Ten_giong':'so_lo'}, inplace=True)

In [19]:
gk.columns

Index(['so_lo', 'kl_truoc_che_bien', 'kl_thanh_pham', 'TL_thanh_pham', 'klpp1',
       'kl_pp2', 'kl_pp3', 'kl_hao_hut', 'TL_hao_hut'],
      dtype='object')

In [20]:
gk1 =gk.reset_index(level=['NM','Ten_giong'])
gk1['stt'] =gk1['NM'].copy()

In [21]:
NM = ['Định Thành','Tà Đảnh','Lương An Trà','Tân Hồng','Động Cát']
ID_NM = [1,2,3,4,5]
stt_dic = dict(zip(NM, ID_NM))

In [22]:
gk1['stt'] =gk1['stt'].map(stt_dic)

In [23]:
gk1 = gk1.sort_values('stt')

# Ghi data vao file excel mau bao cao

In [24]:
data.columns = ['STT', 'NM', 'Ten_giong', 'Cap giong', 'MSL', 'ngay_nhap_kho',
       'ngay_che_bien', 'kl_truoc_che_bien', 'kl_thanh_pham', 'TiLe_TP',
       'klpp1', 'tl_pp1', 'kl_pp2', 'tl_pp2', 'kl_pp3', 'tl_pp3', 'kl_hao_hut',
       'tl_hao_hut', 'note', 'WEEK', 'month','ID']

In [25]:
out_put = os.path.join(duong_dan,"data_out_put.xlsx")


In [26]:
import xlwings as xw

In [27]:
wb =xw.Book("BC_chebien.xlsx")

In [28]:
sht = wb.sheets('data')

In [29]:
# lr = wb.sheets('data').range('A' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
lr = sht.range('A' + str(wb.sheets[0].cells.last_cell.row)).end('up').row +1
lr2 = wb.sheets('Sumary').range('E' + str(wb.sheets[1].cells.last_cell.row)).end('up').row +2

In [31]:
wb.sheets('Sumary').range(f'A{lr2} : O80').clear()

In [32]:
wb.sheets('Sumary').range(f'B{lr2}').options(pd.DataFrame, index=False).value = gk1

In [33]:
sht.range(f'3:{lr}').delete()

In [34]:
sht.range('a3').value = data.iloc[:,1:]

In [35]:
sht.range('3:3').delete()

In [36]:
data.to_excel(out_put)

In [37]:
lr3 = sht[f'A{sht.cells.last_cell.row}'].end('up').row
lr3

16

In [38]:

sht['j2'].api.AutoFill(sht[f'j2:j{lr3}'].api, AutoFillType.xlFillDefault)

True

In [39]:
sht['L2'].api.AutoFill(sht[f'L2:L{lr3}'].api, AutoFillType.xlFillDefault)
sht['N2'].api.AutoFill(sht[f'N2:N{lr3}'].api, AutoFillType.xlFillDefault)
sht['P2:R2'].api.AutoFill(sht[f'P2:R{lr3}'].api, AutoFillType.xlFillDefault)
sht['W2:X2'].api.AutoFill(sht[f'W2:X{lr3}'].api, AutoFillType.xlFillDefault)

True

In [40]:
sht.range(f't2:u{lr3}').clear()

In [41]:
sht.range(f't3:t{lr3}').value = 'W'+ w_num

In [42]:
sht.range(f'u3:u{lr3}').value = d_thang

In [43]:
my_txt = f'Tuần {w_num} ({day_of_last_week}-{month_of_last_week} đến {to_day.day}-{d_thang})'


In [44]:
# print(f'Tuần {w_num} (từ {day_of_last_week}-{d_thang} đến {to_day.day}-{d_thang})')

In [45]:
wb.sheets('BC_THEO NM').range('a6').value = my_txt


l1 = data['ID'].unique()
m = len(l1)


wb.sheets[3].range("b9:b49").clear()

wb.sheets[3].range("b9").value = l1.reshape(m,1)

n = m+8
wb.sheets[3].range('C9:N9').api.AutoFill(wb.sheets[3].range(f'C9:N{n}').api, AutoFillType.xlFillDefault)

In [46]:
wb.save()

In [47]:
print('Done')

Done
